In [11]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from tensorflow.keras import models, layers, optimizers


2023-03-16 22:46:13.192811: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-16 22:46:13.439424: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-16 22:46:13.439447: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-16 22:46:14.066511: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [12]:
# Define the parameters for the simulated dataset
n_curves = 10000
curve_length = 1000
period_range = [20, 200]
depth_range = [0.01, 0.1]
duration_range = [0.01, 0.1]
stellar_radius_range = [0.5, 2.0]
stellar_mass_range = [0.5, 2.0]
noise_level_range = [0.001, 0.01]


In [9]:
def generate_transit_curve(Rs, Rp, P, i, a, t0, b, Te, feh, logg, sigma, n=1000):
    """
    Generates a simulated transit curve using batman package.

    Parameters:
        Rs (float): Stellar radius (in solar units).
        Rp (float): Planetary radius (in Jupiter units).
        P (float): Planetary orbital period (in days).
        i (float): Orbital inclination (in degrees).
        a (float): Semi-major axis (in AU).
        t0 (float): Transit midpoint (in BJD).
        b (float): Impact parameter.
        Te (float): Stellar effective temperature (in K).
        feh (float): Stellar metallicity ([Fe/H]).
        logg (float): Stellar surface gravity (in cgs).
        sigma (float): Observational noise (in ppm).
        n (int, optional): Number of data points to generate. Default is 1000.

    Returns:
        time (array): Array of time values (in BJD).
        flux (array): Array of flux values.
    """
    # Set up transit parameters
    params = batman.TransitParams()
    params.t0 = t0                      # Transit midpoint
    params.per = P                      # Orbital period
    params.rp = Rp                      # Planet radius
    params.a = a                        # Semi-major axis (in units of stellar radius)
    params.inc = i                      # Orbital inclination (in degrees)
    params.ecc = 0.                     # Eccentricity
    params.w = 90.                      # Argument of periastron
    params.u = [0.1, 0.3]               # Limb darkening coefficients
    params.limb_dark = "quadratic"      # Limb darkening model

    # Set up stellar parameters
    star = batman.Star()
    star.teff = Te                      # Effective temperature (in K)
    star.logg = logg                    # Surface gravity (in cgs)
    star.feh = feh                      # Metallicity ([Fe/H])
    star.radius = Rs                    # Stellar radius (in solar units)

    # Generate time array
    time = np.linspace(t0 - 0.05*P, t0 + 0.05*P, n)
    
    # Generate transit model
    m = batman.TransitModel(params, time)
    flux = m.light_curve(star)

    # Add observational noise
    noise = np.random.normal(loc=0, scale=sigma*1e-6, size=n)
    flux += noise

    return time, flux


In [10]:
np.random.seed(42)

# Set up parameter ranges
Rs_range = (0.5, 2.0)                  # Solar radii
Rp_range = (0.1, 2.0)                  # Jupiter radii
P_range = (1, 10)                      # Days
i_range = (85, 95)                     # Degrees
a_range = (5, 20)                      # AU
t0_range = (2459000, 2459500)          # BJD
b_range = (0, 1)                       # Impact parameter
Te_range = (5000, 6000)                # K
feh_range = (-0.5, 0.5)                # dex
logg_range = (3.5, 4.5)                # cgs
sigma_range = (10, 100)                # ppm

# Generate positive examples
n_pos = 1000
pos_params = np.zeros((n_pos, 12))
pos_params[:, 0] = np.random.uniform(*Rs_range, size=n_pos)
pos_params[:, 1] = np.random.uniform(*Rp_range, size=n_pos)
pos_params[:, 2] = np.random.uniform(*P_range, size=n_pos)
pos_params[:, 3] = np.random.uniform(*i_range, size=n_pos)
pos_params[:, 4] = np.random.uniform(*a_range, size=n_pos)
pos_params[:, 5] = np.random.uniform(*t0_range, size=n_pos)
pos_params[:, 6] = np.random.uniform(*b_range, size=n_pos)
pos_params[:, 7] = np.random.uniform(*Te_range, size=n_pos)
pos_params[:, 8] = np.random.uniform(*feh_range, size=n_pos)
pos_params[:, 9] = np.random.uniform(*logg_range, size=n_pos)
pos_params[:, 10] = np.random.uniform(*sigma_range, size=n_pos)
pos_params[:, 11] = 1
pos_curves = [generate_transit_curve(*params) for params in pos_params]

# Generate negative examples
n_neg = 1000
neg_params = np.zeros((n_neg, 12))
neg_params[:, 0] = np.random.uniform(*Rs_range, size=n_neg)
neg_params[:, 1] = np.random.uniform(*Rp_range, size=n_neg)
neg_params[:, 2] = np.random.uniform(*P_range, size=n_neg)
neg_params[:, 3] = np.random.uniform(*i_range, size=n_neg)
neg_params[:, 4] = np.random.uniform(*a_range, size=n_neg)
neg_params[:, 5] = np.random.uniform(*t0_range, size=n_neg)
neg_params[:, 6] = np.random.uniform(*b_range, size=n_neg)
neg_params[:, 7] = np.random.uniform(*Te_range, size=n_neg)
neg_params[:, 8] = np.random.uniform(*feh_range, size=n_neg)
neg_params[:, 9] = np.random.uniform(*logg_range, size=n_neg)
neg_params[:, 10] = np.random.uniform(*sigma_range, size=n_neg)
neg_params[:, 11] = 0
neg_curves = [generate_transit_curve(*params) for params in neg_params]

AttributeError: module 'batman' has no attribute 'Star'